# Artifacts Evaluation Instructions: #506   Combining Static and Dynamic Code Information for Software Vulnerability Prediction

## Preliminaries

This interactive Jupyter notebook provides a small-scale demo to showcase the program representation, vulnerability detection, and prediction of vulnerability detection discussed in the paper.

The main results of our CCS 2023 paper involve comparing the performance of our vulnerability detection with prior machine learning-based approaches. The evaluation presented in our paper was conducted on a much larger dataset and for a longer duration. The intention of this notebook is to provide minimal working examples that can be evaluated within a reasonable time frame.

## Instructions for Experimental Workflow:

Before you start, please first make a copy of the notebook by going to the landing page. Then select the checkbox next to the notebook titled *main.ipynb*, then click "**Duplicate**".

Click the name of the newly created Jupyter Notebook, e.g. **AE-Copy1.ipynb**. Next, select "**Kernel**" > "**Restart & Clear Output**". Then, repeatedly press the play button (the tooltip is "run cell, select below") to step through each cell of the notebook.

Alternatively, select each cell in turn and use "**Cell**"> "**Run Cell**" from the menu to run specific cells. Note that some cells depend on previous cells being executed. If any errors occur, ensure all previous cells have been executed.

## Important Notes

**Some cells can take more than half an hour to complete; please wait for the results until step to the next cell.** 

High load can lead to a long wait for results. This may occur if multiple reviewers are simultaneously trying to generate results. 

The experiments are customisable as the code provided in the Jupyter Notebook can be edited on the spot. Simply type your changes into the code blocks and re-run using **Cell > Run Cells** from the menu.

## Links to The Paper

For each step, we note the section number of the submitted version where the relevant technique is described or data is presented.

The main results are presented in Figures 9-12 of the submitted paper.

# Demo 1: The Concoction Model Architecture

This demo corresponds to the architecture given in Figure 3. Note that This is a small-scale demo for vulnerability detection. The full-scale evaluation used in the paper takes over 24 hours to run.

## Step 1. Program representation

The program representation component maps the input source code and dynamic symbolic execution traces of the target function into a numerical embedding vector.

#### *Static representation model*:

In [18]:
#  Extract Program Information (To show static code information like sec 3.3.1.)
import os
current=os.getcwd()
dir=os.path.join(current,'feature/static/github')
projectPath=os.path.join(current,'example/jasper-version-1.900.1')
!cd $dir && python GetStatic.py $projectPath

/home
2023-10-08 13:23:42,036 - INFO - project : /home/feature/static/test_pro
2023-10-08 13:23:42,036 - INFO - start to get function segmentation... 
finished:0.00%   Batch 1
2023-10-08 13:23:43,303 - INFO - start to get graphRelation by joern... 
joern_relationgood.py over...
2023-10-08 13:24:03,708 - INFO - start to get seven edges...
2023-10-08 13:24:06,910 - INFO - start to get cfgpath  by joern and concate... 
Processing getcfg jasper.c-testtttt.c: 100%|██████| 2/2 [00:24<00:00, 12.34s/it]
2023-10-08 13:24:32,523 - INFO - extracted static feature stored in : /home/feature/static/github/static/


In [2]:
#  Pretrain Representation Models (Here we use Graphcodebert like sec 3.4)
#preprocess
import os
print(os.getcwd())
current=os.getcwd()
path=os.path.join(current,'concoction/data/dataset')
outputPath=os.path.join(current,'concoction/data/output_static.txt')
dir=os.path.join(current,'concoction/pretrainedModel/staticRepresentation')
!cd $dir && python preprocess.py --data_path $path --output_path $outputPath
#pretrain
storedPath=os.path.join(os.getcwd(),'concoction/pretrainedModel/staticRepresentation/trainedModel')
!cd $dir && python train.py --model_name_or_path graphcodebert-base --train_data_file $outputPath --per_device_train_batch_size 8 --do_train --output_dir $storedPath --mlm --overwrite_output_dir --line_by_line

/home
dirs: 276it [00:02, 95.13it/s] 
10/10/2023 08:18:27 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 3, distributed training: False, 16-bits training: False
10/10/2023 08:18:27 - INFO - __main__ - Training/evaluation parameters TrainingArguments(output_dir=/home/concoction/pretrainedModel/staticRepresentation/trainedModel, overwrite_output_dir=True, do_train=True, do_eval=False, do_predict=False, evaluation_strategy=EvaluationStrategy.NO, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=3.0, max_steps=-1, lr_scheduler_type=SchedulerType.LINEAR, warmup_steps=0, logging_dir=runs/Oct10_08-18-27_10290d7ddecb, logging_first_step=False, logging_steps=500, save_steps=500, save_total_limit=None, no_cuda=False, seed=42, fp16=False, fp16_opt_level=O


  0%|                                                    | 0/36 [00:04<?, ?it/s]


In [6]:
#  Show the Trained Model (like the example in https://github.com/microsoft/CodeBERT)
from transformers import AutoTokenizer, AutoModel
import torch
storedPath=os.path.join(os.getcwd(),'concoction/pretrainedModel/staticRepresentation/trainedModel')
tokenizer = AutoTokenizer.from_pretrained(storedPath)
model = AutoModel.from_pretrained(storedPath)
nl_tokens=tokenizer.tokenize("return maximum value")
code_tokens=tokenizer.tokenize("def max(a,b): if a>b: return a else return b")
tokens=[tokenizer.cls_token]+nl_tokens+[tokenizer.sep_token]+code_tokens+[tokenizer.eos_token]
tokens_ids=tokenizer.convert_tokens_to_ids(tokens)
context_embeddings=model(torch.tensor(tokens_ids)[None,:])[0]
print(context_embeddings)

Some weights of RobertaModel were not initialized from the model checkpoint at /home/concoction/pretrainedModel/staticRepresentation/trainedModel and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tensor([[[ 0.2573,  0.2688, -0.1238,  ..., -0.1123, -0.1479,  0.3731],
         [ 0.5010, -0.2858,  0.6845,  ...,  0.2044,  0.8592,  0.0892],
         [ 0.1018,  0.6084,  0.0398,  ...,  0.5097, -0.3028,  0.1666],
         ...,
         [ 0.2921,  0.1633,  0.7146,  ...,  0.7883,  0.5627, -0.1219],
         [ 0.2624,  0.6262, -0.0925,  ...,  1.7611, -0.5254,  0.1670],
         [ 0.2574,  0.2688, -0.1238,  ..., -0.1123, -0.1481,  0.3735]]],
       grad_fn=<NativeLayerNormBackward>)



#### *Dynamic representation model*:


In [ ]:
#  Extract Program Information (To show dynamic code information like sec 3.3.2.)
import os
print(os.getcwd())
current=os.getcwd()
dir=os.path.join(current,'feature/dynamic')
shell_file=os.path.join(dir,'main.sh')
project_path=os.path.join(current,'jasper-version-1.900.1/jasper-version-1.900.1')
dependency_txt=os.path.join(dir,'before_insert.txt')
compile_txt=os.path.join(dir,'compile.txt')
kleecmd_txt=os.path.join(dir,'do.txt')
!cd $dir && sh $shell_file $project_path $dependency_txt $compile_txt $kleecmd_txt 

In [7]:
#  Pretrain Representation Models (Here we use Simcse like sec 3.4)
#preprocess
import os
print(os.getcwd())
current=os.getcwd()
path=os.path.join(current,'concoction/data/feature')
outputPath=os.path.join(current,'concoction/data/output_dynamic.txt')
dir=os.path.join(current,'concoction/pretrainedModel/dynamicRepresentation')
!cd $dir && python preprocess.py --data_path $path --output_path $outputPath
#training
! cd $dir && python train.py --model_name_or_path bert-base-uncased     --train_file $outputPath   --output_dir ./result    --num_train_epochs 1     --per_device_train_batch_size 32     --learning_rate 3e-5     --max_seq_length 32      --metric_for_best_model stsb_spearman  --load_best_model_at_end     --eval_steps 2     --pooler_type cls     --mlp_only_train     --overwrite_output_dir     --temp 0.05     --do_train

/home
dirs: 12it [00:00, 31.31it/s]
cpu
device
09/20/2023 08:41:12 - WARNING - __main__ -   Process rank: -1, device: cpu, n_gpu: 0 distributed training: False, 16-bits training: False
09/20/2023 08:41:12 - INFO - __main__ -   Training/evaluation parameters OurTrainingArguments(output_dir='./result', overwrite_output_dir=True, do_train=True, do_eval=False, do_predict=False, evaluation_strategy=<EvaluationStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=32, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=3e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=1.0, max_steps=-1, lr_scheduler_type=<SchedulerType.LINEAR: 'linear'>, warmup_steps=0, logging_dir='runs/Sep20_08-41-12_10290d7ddecb', logging_first_step=False, logging_steps=500, save_steps=500, save_total_limit=None, no_cuda=Fals

In [8]:
#  Show the Trained Model (like the example in https://github.com/microsoft/CodeBERT)
from transformers import AutoTokenizer, AutoModel
import torch,os
storedPath=os.path.join(os.getcwd(),'concoction/pretrainedModel/dynamicRepresentation/result')
tokenizer = AutoTokenizer.from_pretrained(storedPath)
model = AutoModel.from_pretrained(storedPath)

nl_tokens=tokenizer.tokenize("return maximum value")
code_tokens=tokenizer.tokenize("def max(a,b): if a>b: return a else return b")
tokens=[tokenizer.cls_token]+nl_tokens+[tokenizer.sep_token]+code_tokens
tokens_ids=tokenizer.convert_tokens_to_ids(tokens)
context_embeddings=model(torch.tensor(tokens_ids)[None,:])[0]
print(context_embeddings)

Some weights of BertModel were not initialized from the model checkpoint at /home/concoction/pretrainedModel/dynamicRepresentation/result and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tensor([[[-0.3348, -0.7194, -0.3097,  ..., -0.6366, -0.3357,  1.0084],
         [-0.0132, -0.1685,  0.1986,  ..., -0.1092, -0.2491,  0.3447],
         [-0.3284, -0.2352,  0.0701,  ..., -0.6065, -0.3233,  0.3300],
         ...,
         [-0.2394, -0.2376,  0.3637,  ..., -0.2333,  0.3245,  0.2509],
         [ 0.2682,  0.0495,  0.9866,  ..., -0.1620, -0.9476,  0.1886],
         [ 0.7464,  0.2502,  0.0631,  ..., -1.2616,  0.3442,  0.3946]]],
       grad_fn=<NativeLayerNormBackward>)




## Step 2. Vulnerability Detection

 Concoction’s detection component takes the joint embedding as input to predict the presence of vulnerabilities.  Our current implementation only identifies whether a function may contain a vulnerability or bug and does not specify the type of vulnerability. Here we use SARD benchmarks.

**approximate runtime ~ 30 minutes (please wait before moving to the next cell)**

#### *Dynamic representation model*:


In [26]:
#  Train detection Model (Here we use Simcse like sec 3.5.2)
import os
print(os.getcwd())
current=os.getcwd()
path=os.path.join(current,'concoction/data/feature')
dir=os.path.join(current,'concoction/detectionModel')
!cd $dir && python evaluation_bug.py --path_to_data /home/model/data/sard/cwe416 --mode train


/home/CONCOCTION
0
2023-09-12 06:45:51,369 : Load data from exist npy
[2023-09-12 06:45:51] INFO (root/MainThread) Load data from exist npy
total train files is  146
2023-09-12 06:45:51,529 : Computing embedding for train
[2023-09-12 06:45:51] INFO (root/MainThread) Computing embedding for train
100%|█████████████████████████████████████████████| 3/3 [00:07<00:00,  2.52s/it]
2023-09-12 06:45:59,099 : Computed train embeddings
[2023-09-12 06:45:59] INFO (root/MainThread) Computed train embeddings
2023-09-12 06:45:59,099 : Computing embedding for dev
[2023-09-12 06:45:59] INFO (root/MainThread) Computing embedding for dev
100%|█████████████████████████████████████████████| 1/1 [00:02<00:00,  2.49s/it]
2023-09-12 06:46:01,588 : Computed dev embeddings
[2023-09-12 06:46:01] INFO (root/MainThread) Computed dev embeddings
2023-09-12 06:46:01,589 : Computing embedding for test
[2023-09-12 06:46:01] INFO (root/MainThread) Computing embedding for test
100%|██████████████████████████████████████

2023-09-12 06:46:04,415 : Training for 50 epochs: f1 = 0.9795917957735685, precision = 0.9599999785423279,recall = 1.0,accuracy = 0.9795918367346939
[2023-09-12 06:46:04] INFO (root/MainThread) Training for 50 epochs: f1 = 0.9795917957735685, precision = 0.9599999785423279,recall = 1.0,accuracy = 0.9795918367346939
2023-09-12 06:46:04,434 : Training for 60 epochs: f1 = 0.9795917957735685, precision = 0.9599999785423279,recall = 1.0,accuracy = 0.9795918367346939
[2023-09-12 06:46:04] INFO (root/MainThread) Training for 60 epochs: f1 = 0.9795917957735685, precision = 0.9599999785423279,recall = 1.0,accuracy = 0.9795918367346939
2023-09-12 06:46:04,452 : Training for 70 epochs: f1 = 0.9795917957735685, precision = 0.9599999785423279,recall = 1.0,accuracy = 0.9795918367346939
[2023-09-12 06:46:04] INFO (root/MainThread) Training for 70 epochs: f1 = 0.9795917957735685, precision = 0.9599999785423279,recall = 1.0,accuracy = 0.9795918367346939
2023-09-12 06:46:04,470 : Training for 80 epochs:

In [9]:
#  Show the Trained Model (Load trained model and test on test case)
import os
print(os.getcwd())
current=os.getcwd()
path=os.path.join(current,'concoction/data/dataset')
dir=os.path.join(current,'concoction/detectionModel')
!cd $dir && python evaluation_bug.py  --model_to_load $dir/f1_0.9795917957735685_2023-09-12.h5 --path_to_data $path  --mode test

/home
{'nhid': 2, 'optim': 'adam', 'batch_size': 64, 'tenacity': 5, 'epoch_size': 200}
0
2023-09-20 08:58:23,236 : Load data from exist npy
[2023-09-20 08:58:23] INFO (root/MainThread) Load data from exist npy
100%|███████████████████████████████████████████| 32/32 [01:39<00:00,  3.10s/it]
2023-09-20 09:00:02,732 : Computed data embeddings
[2023-09-20 09:00:02] INFO (root/MainThread) Computed data embeddings
2023-09-20 09:00:02,733 : model load from/home/concoction/detectionModel/f1_0.9795917957735685_2023-09-12.h5
[2023-09-20 09:00:02] INFO (root/MainThread) model load from/home/concoction/detectionModel/f1_0.9795917957735685_2023-09-12.h5
2023-09-20 09:00:02,961 : prediction : f1 = 0.6939571400480062, precision = 0.5313432812690735,recall = 1.0,accuracy = 0.5389133627019089
[2023-09-20 09:00:02] INFO (root/MainThread) prediction : f1 = 0.6939571400480062, precision = 0.5313432812690735,recall = 1.0,accuracy = 0.5389133627019089
this is the data path: /home/concoction/data/dataset


## Step 3. Deployment

This demo shows how to deploy our trained model on a real world project. Here we apply the xx as our test project.

#### *Path Selection for Symbolic Execution*:
After training the end-to-end model, we develop a path selection component to automatically select a subset of important paths whose dynamic traces are likely to improve prediction accuracy during deployment.

*approximate runtime ~ 30 minutes*

In [13]:
# Execution path representation (shown as Sec. 3.6.1)
import os
print(os.getcwd())
current=os.getcwd()
path=os.path.join(current,'concoction/data/dataset')
storedDir=os.path.join(current,'concoction/data/feature_path')
dir=os.path.join(current,'concoction/pathSelection')
if not os.path.exists(storedDir):
    os.mkdir(storedDir)
! cd $dir && python preprocess.py --data_path $path --stored_path $storedDir
print(f"Execution path representation stored in {storedDir}")

/home
preprocess data........
dirs: 0it [00:00, ?it/s]/home/concoction/data/dataset/jas_icc.c-jas_iccgetsint32.c.txt
dirs: 1it [00:00,  7.22it/s]/home/concoction/data/dataset/jas_malloc.c-jas_free.c.txt
/home/concoction/data/dataset/jas_icc.c-jas_iccgetuint.c.txt
dirs: 3it [00:00,  9.78it/s]/home/concoction/data/dataset/jpc_mqenc.c-jpc_mqenc_codelps.c.txt
/home/concoction/data/dataset/jpc_enc.c-prc_create.c.txt
dirs: 5it [00:00, 11.89it/s]/home/concoction/data/dataset/jp2_cod.c-jp2_box_create.c.txt
/home/concoction/data/dataset/jas_image.c-jas_image_writecmpt.c.txt
dirs: 7it [00:08,  1.70s/it]/home/concoction/data/dataset/jpc_qmfb.c-jpc_qmfb_split_row.c.txt
dirs: 8it [00:09,  1.53s/it]/home/concoction/data/dataset/jas_stream.c-jas_stream_flushbuf.c.txt
dirs: 9it [00:09,  1.23s/it]/home/concoction/data/dataset/jas_tmr.c-jas_tmr_start.c.txt
/home/concoction/data/dataset/jpc_mqenc.c-jpc_mqenc_flush.c.txt
dirs: 11it [00:09,  1.26it/s]/home/concoction/data/dataset/jpc_cs.c-jpc_siz_destroypa

dirs: 98it [00:41,  2.21it/s]/home/concoction/data/dataset/jas_cm.c-jas_clrspc_numchans.c.txt
dirs: 99it [00:41,  2.56it/s]/home/concoction/data/dataset/jpc_t2cod.c-jpc_pi_create0.c.txt
dirs: 100it [00:41,  2.99it/s]/home/concoction/data/dataset/jas_stream.c-mem_close.c.txt
dirs: 101it [00:41,  3.33it/s]/home/concoction/data/dataset/jas_cm.c-jas_cmshapmat_invmat.c.txt
/home/concoction/data/dataset/jas_seq.c-jas_matrix_create.c.txt
dirs: 103it [00:42,  3.30it/s]/home/concoction/data/dataset/pnm_cod.c-pnm_fmt.c.txt
dirs: 104it [00:42,  3.78it/s]/home/concoction/data/dataset/jpc_mqenc.c-jpc_mqenc_setctxs.c.txt
/home/concoction/data/dataset/jpc_enc.c-pass_destroy.c.txt
/home/concoction/data/dataset/jpc_t2cod.c-jpc_pi_init.c.txt
dirs: 107it [00:42,  6.33it/s]/home/concoction/data/dataset/jas_stream.c-mem_seek.c.txt
/home/concoction/data/dataset/jpc_tagtree.c-jpc_tagtree_reset.c.txt
dirs: 109it [00:42,  5.87it/s]/home/concoction/data/dataset/jpc_t1enc.c-jpc_encclnpass.c.txt
dirs: 110it [00:4

In [30]:
# Active learning for path selection (Sec. 3.6.2)
#data preprocess

import os
print(os.getcwd())
current=os.getcwd()
path=os.path.join(current,'concoction/data/feature')
storedDir=os.path.join(current,'concoction/data/feature_path')
final_storedDir=os.path.join(current,'concoction/data/feature_path_text')
dir=os.path.join(current,'concoction/pathSelection')
if not os.path.exists(storedDir):
    os.mkdir(storedDir)
# ! cd $dir && python preprocess.py --data_path $path --stored_path $storedDir
#select path
!cd $dir && python train.py --data_path $storedDir --stored_path $final_storedDir

/home
Load Data....
100%|█████████████████████████████████████████| 126/126 [00:16<00:00,  7.64it/s]
saved the select path to /home/concoction/data/feature_path_text


In [2]:
# Symbolic execution for chosen paths (Sec. 3.6.3)
import os
current_folder = os.path.abspath('.')
current=current_folder
path=os.path.join(current,'feature/dynamic/map.py')
dataset=os.path.join(current,'concoction/data/dataset')
feature_path_text=os.path.join(current,'concoction/data/feature_path_text')
final_path=os.path.join(current,'concoction/data/feature_result')
!python $path $feature_path_text $dataset $final_path

stored in the /homee/concoction/data/feature_result


#### *Fuzzing for Test Case Generation*:

 We use fuzzing techniques to generate test cases for functions predicted to contain potential vulnerabilities, aiming to automate the testing process and minimize the need for manual inspection.

In [ ]:
# Utilizing AFL++ To Objective project (Shown as Sec. 3.7)

## Demo 2: Experimental Evaluation

Here, we provide a small-sized evaluation to showcase the working mechanism of Concoction bug detection. A full-scale evaluation, which takes more than a day to run, is provided through the Docker image (with detailed instructions on our project Github).

### Large-scale Testing (Section 5.1)

 This part (add a web link) gives a quantified summary of Concoction for detecting function-level code vulnerabilities across the 20 projects listed in Table 1 in our papaer.

This demo corresponds to Table 5 of the submitted manuscript.


### Evaluation on Open Dataset (Section  5.2)

We now evaluate our vulnerability detection model on the SARD and CVE datasets in Table 2 in paper.

This demo corresponds to Figure 9 and 10 of the submitted manuscript.

*approximate runtime = 10 minutes for one benchmark*

#### *Sard Dataset*:


In [1]:
# Prepare the dataset and preprocess (use parameter to change CWE type and method,
#                                     like FUN A( dataset = 'CWE-123', method = 'Vuldeepecker' ))
from demo2.demo2 import *
# Train and test.
EvaluationSard("CWE-416","vuldeepecker").run()

/root/anaconda3/envs/vuldeepecker1/bin/python vuldeepecker.py --data_path /home/ExperimentalEvaluation/data/sard/cwe416 --mode train


Using TensorFlow backend.
Token indices sequence length is longer than the specified maximum sequence length for this model (625 > 512). Running this sequence through the model will result in indexing errors
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
2023-12-05 22:01:16.289410: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2023-12-05 22:01:16.372572: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: NVIDIA GeForce RTX 2080 Ti major: 7 minor: 5 memoryClockRate(GHz): 1.635
pciBusID: 0000:19:00.0
2023-12-05 22:01:16.372975: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 1 with properties: 
name: NVIDIA GeForce RTX 2080 Ti major: 7 minor: 5 memoryClockRate(GHz): 1.635
pciBusID: 0000:1b:00.0
2023-12-05 22:01:16.373305: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 2 with properties: 
name: NVIDIA

==============random_state===========
 42
tracking <tf.Variable 'Variable:0' shape=() dtype=int32> tp
tracking <tf.Variable 'Variable_1:0' shape=() dtype=int32> fp
tracking <tf.Variable 'Variable_2:0' shape=() dtype=int32> tp
tracking <tf.Variable 'Variable_3:0' shape=() dtype=int32> fn
tracking <tf.Variable 'Variable_4:0' shape=() dtype=int32> tp
tracking <tf.Variable 'Variable_5:0' shape=() dtype=int32> fp
tracking <tf.Variable 'Variable_6:0' shape=() dtype=int32> tp
tracking <tf.Variable 'Variable_7:0' shape=() dtype=int32> fn
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 600)               842400    
_________________________________________________________________
dense_1 (Dense)              (None, 300)               180300    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 300)      

#### *Github Dataset*:


In [2]:
from demo2.demo2 import *
EvaluationGithub("Github","vuldeepecker").run()

/root/anaconda3/envs/vuldeepecker1/bin/python vuldeepecker.py --data_path /home/ExperimentalEvaluation/data/github_0.65 --mode train


Using TensorFlow backend.
Token indices sequence length is longer than the specified maximum sequence length for this model (9950 > 512). Running this sequence through the model will result in indexing errors
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
2023-12-05 22:03:11.931644: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2023-12-05 22:03:11.998739: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: NVIDIA GeForce RTX 2080 Ti major: 7 minor: 5 memoryClockRate(GHz): 1.635
pciBusID: 0000:19:00.0
2023-12-05 22:03:11.999050: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 1 with properties: 
name: NVIDIA GeForce RTX 2080 Ti major: 7 minor: 5 memoryClockRate(GHz): 1.635
pciBusID: 0000:1b:00.0
2023-12-05 22:03:11.999320: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 2 with properties: 
name: NVIDI

==============random_state===========
 42
tracking <tf.Variable 'Variable:0' shape=() dtype=int32> tp
tracking <tf.Variable 'Variable_1:0' shape=() dtype=int32> fp
tracking <tf.Variable 'Variable_2:0' shape=() dtype=int32> tp
tracking <tf.Variable 'Variable_3:0' shape=() dtype=int32> fn
tracking <tf.Variable 'Variable_4:0' shape=() dtype=int32> tp
tracking <tf.Variable 'Variable_5:0' shape=() dtype=int32> fp
tracking <tf.Variable 'Variable_6:0' shape=() dtype=int32> tp
tracking <tf.Variable 'Variable_7:0' shape=() dtype=int32> fn
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 600)               842400    
_________________________________________________________________
dense_1 (Dense)              (None, 300)               180300    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 300)      


#### Full-scale evaluation data

We now plot the diagrams using full-scale evaluation data (it would take too long to run the experiment lively). The results correspond to Figure 9 and 10 (Section 5.2) of the submitted manuscript.

In [ ]:
# Output Figure 9: Evaluation on standard vulnerability databases. Min-max bars show performance across vulnerability types.
# Output Figure 10:  Evaluation on the CVE dataset Concoction gives the best performance across evaluation metrics.

### Case Study 3:  Evaluation on Opensource Projects (Section 5.3)

We now compare to the baseline methods by applying them to the three open-source projects in Table 3 with a total of 35 CVEs reported by independent users.

This demo corresponds to Figure 11 of the submitted manuscript.

*approximate runtime = xx minutes for one benchmark*

### Client RL Deployment Demo

This demo shows how to apply the saved client RL to optimize a test program for Code Size Reduction. 

*approximate runtime ~ 15 minutes*

#### Performance evaluation on benchmarks
Benchmarks: Sqlite, Libtiff, Libpng.
*approximate runtime ~ 20 minutes*

In [ ]:
# Load model and test benchmarks (Input benchmark and method like FUN A( project = 'Sqlite', method = 'Concoction').

#### Full-scale evaluation data
We now generate the table using full-scale evaluation data (it would take too long to run the experiment lively). The results correspond to Figure 11 (Section 5.3) of the submitted manuscript.

In [ ]:
# Output Figure 11: The number of vulnerabilities identified Concoction and other methods for open-source projects in Table 3.

### Further Analysis (Alternative)

####  DL model implementation choices.

we evaluate several variants of Concoction on the CVE dataset.

This demo corresponds to Figure 12 of the submitted manuscript.

*approximate runtime = xx minutes for one benchmark*

In [ ]:
# Load model and test CVE benchmarks (Input method like FUN A( method = 'Static').

#### Full-scale evaluation data
We now generate the figure using full-scale evaluation data (it would take too long to run the experiment lively). The results correspond to Figure 12 (Section 5.4) of the submitted manuscript.


In [ ]:
# Output Figure 12:  Comparing implementation variants of Concoction on the CVE dataset.

/root/anaconda3/envs/pytorch1.7.1/lib/python3.6/site-packages
